# Stats Project!
---
This dataset catalogues human genetic variants (Also called Allele)
- There are many genes within the human body
- Gene copying is not perfect, so genes in different cells may differ a lot from each other
- Some of these variants are safe, some are dangerous
- Some variants are more common than others, and this dataset likely contains most of the common ones
- The SIFT, PolyPhen, etc. scores are generated by genome-analysing algorithms that can identify whether the variant is dangerous (deleterious / damaging / pathogenic) or safe (benign / tolerated)

The dataset in this repository is not the same as the [original](https://www.kaggle.com/datasets/kevinarvai/clinvar-conflicting), the columns were renamed to be more descriptive and some useless columns were removed

Hypothesis: **Does the statistical properties of a gene expression help predict whether or not it is dangerous?**

### Installs and imports

In [2]:
%pip install numpy pandas seaborn
import numpy as np
import pandas as pd
import seaborn as sns

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 706.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 656.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 711.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 714.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 708.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 701.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 620.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 719.1 kB/s eta 0:00:0000:0100:01


In [7]:
df = pd.read_csv("./Clinvar.csv")
df.describe()

/tmp/ipykernel_178665/2775463472.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Clinvar.csv")


,Position on chromosome,Allele frequencies (GO-ESP),Allele frequencies (ExAC),Allele frequencies (1000 genomes project),Allele origin,Has conflicting submissions,STRAND,LoFtool score,CADD score (Phred-scaled),Raw CADD score,BLOSUM62 score
count,6.518800e+04,65188.000000,65188.000000,65188.000000,65188.000000,65188.000000,65174.000000,60975.000000,64096.000000,64096.000000,25593.000000
mean,7.757594e+07,0.014511,0.014492,0.015263,1.342486,0.252102,-0.006659,0.345058,15.685616,2.554131,-0.402258
std,5.974051e+07,0.057795,0.059542,0.059527,5.688772,0.434223,0.999985,0.361238,10.836350,2.961553,1.872684
min,9.610000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000069,0.001000,-5.477391,-3.000000
25%,3.254179e+07,0.000000,0.000000,0.000000,1.000000,0.000000,-1.000000,0.024300,7.141000,0.462951,-2.000000
50%,5.797021e+07,0.000000,0.000040,0.000000,1.000000,0.000000,-1.000000,0.157000,14.090000,1.642948,-1.000000
75%,1.127454e+08,0.001200,0.001230,0.001600,1.000000,1.000000,1.000000,0.710000,24.100000,4.381392,1.000000
max,2.476080e+08,0.499000,0.499890,0.499800,513.000000,1.000000,1.000000,1.000000,99.000000,46.556261,3.000000
